<a href="https://colab.research.google.com/github/saito1111/Speaker-diarization-/blob/main/Enhanced_Diarization_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ Enhanced Multi-Channel Speaker Diarization Training

**Objectif :** Entraîner un système de diarization de locuteurs avancé sur le corpus AMI  
**Architecture :** TCN multi-échelle avec attention, classification de locuteurs et gestion mémoire optimisée

---

## 📋 Table des Matières
1. [Configuration de l'Environnement](#setup)
2. [Téléchargement du Corpus AMI](#data)
3. [Préparation des Données](#preprocessing)
4. [Modèle et Configuration](#model)
5. [Entraînement](#training)
6. [Évaluation](#evaluation)
7. [Sauvegarde et Visualisations](#results)

## 🔧 1. Configuration de l'Environnement

Installation de Conda et des dépendances nécessaires pour l'entraînement.

In [ ]:
# Installation de Conda sur Google Colab
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda --version

import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

In [ ]:
# Création de l'environnement conda pour la diarization
!conda create -n diarization python=3.9 -y
!conda activate diarization

# Installation des dépendances principales via conda
!conda install -n diarization pytorch torchaudio cudatoolkit=11.8 -c pytorch -c nvidia -y
!conda install -n diarization numpy scipy scikit-learn matplotlib seaborn pandas -y

# Activation de l'environnement dans le notebook
import os
os.environ['CONDA_DEFAULT_ENV'] = 'diarization'
os.environ['PATH'] = '/usr/local/envs/diarization/bin:' + os.environ['PATH']

In [ ]:
# Installation des dépendances spécialisées
!pip install wandb optuna tqdm psutil
!pip install librosa soundfile
!pip install speechbrain

# Vérification des installations
import torch
import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

### 📂 Clonage du Répertoire et Configuration

In [ ]:
# Cloner le projet depuis GitHub
import os
from pathlib import Path

# Vérifier si le repository est déjà présent
current_dir = Path.cwd()
repo_name = "Speaker-diarization-"

print(f"📁 Répertoire courant: {current_dir}")

# Si on n'est pas déjà dans le repository, le cloner
if not (current_dir / "src").exists():
    print("📥 Clonage du repository...")
    !git clone https://github.com/saito1111/Speaker-diarization-.git
    
    # Vérifier que le clonage a réussi
    if Path(repo_name).exists():
        print(f"✅ Repository cloné avec succès dans {repo_name}/")
        %cd {repo_name}
        print(f"📂 Changement de répertoire vers: {Path.cwd()}")
    else:
        print("❌ ERREUR: Le clonage a échoué!")
        print("🔧 Vérifiez votre connexion internet et l'URL du repository")
        raise Exception("Clonage du repository échoué")
else:
    print("✅ Repository déjà présent")

# Vérifier la structure du projet
print("\n📋 Vérification de la structure du projet:")
print("📁 Racine du projet:")
!ls -la

print("\n📂 Contenu du répertoire src/:")
if Path("src").exists():
    !ls -la src/
    print("✅ Répertoire src trouvé")
else:
    print("❌ ERREUR: Répertoire src manquant!")
    raise FileNotFoundError("Le répertoire src n'existe pas après le clonage")

print("\n🎉 Structure du projet vérifiée avec succès!")

In [ ]:
# Configuration des chemins et imports
import sys
sys.path.append('./src')

# Créer les dossiers nécessaires
!mkdir -p data/ami_corpus/audio
!mkdir -p data/ami_corpus/annotations
!mkdir -p models/checkpoints
!mkdir -p results/logs
!mkdir -p results/figures

# Variables globales
DATA_DIR = Path('./data/ami_corpus')
AUDIO_DIR = DATA_DIR / 'audio'
ANNOTATION_DIR = DATA_DIR / 'annotations'
MODEL_DIR = Path('./models/checkpoints')
RESULTS_DIR = Path('./results')

print(f"Répertoires configurés:")
print(f"- Audio: {AUDIO_DIR}")
print(f"- Annotations: {ANNOTATION_DIR}")
print(f"- Modèles: {MODEL_DIR}")
print(f"- Résultats: {RESULTS_DIR}")

## 📊 2. Téléchargement et Préparation du Corpus AMI

Le corpus AMI contient des enregistrements de réunions avec annotations temporelles des locuteurs.

In [ ]:
# Téléchargement réel du corpus AMI avec vérification et stockage persistant
# URL officielle: https://groups.inf.ed.ac.uk/ami/corpus/

import urllib.request
import zipfile
from tqdm import tqdm
import hashlib
import shutil
from pathlib import Path

class AMICorpusDownloader:
    """Gestionnaire complet pour le téléchargement et la vérification du corpus AMI."""
    
    def __init__(self, base_dir):
        self.base_dir = Path(base_dir)
        self.audio_dir = self.base_dir / "ami_audio"
        self.annotation_dir = self.base_dir / "ami_annotations"
        self.download_dir = self.base_dir / "downloads"
        
        # Créer les répertoires
        for dir_path in [self.audio_dir, self.annotation_dir, self.download_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
    
    def download_with_progress_bar(self, url, filename, description="Téléchargement"):
        """Télécharge un fichier avec barre de progression détaillée."""
        print(f"🔽 {description}: {url}")
        
        class ProgressBar:
            def __init__(self):
                self.pbar = None
            
            def __call__(self, block_num, block_size, total_size):
                if not self.pbar:
                    self.pbar = tqdm(total=total_size, unit='B', unit_scale=True, desc=description)
                
                downloaded = block_num * block_size
                if downloaded < total_size:
                    self.pbar.update(block_size)
                else:
                    self.pbar.close()
        
        urllib.request.urlretrieve(url, filename, ProgressBar())
        print(f"✅ Téléchargé: {filename}")
    
    def verify_file_integrity(self, filepath, expected_size=None, expected_hash=None):
        """Vérifie l'intégrité d'un fichier téléchargé."""
        if not filepath.exists():
            return False, "Fichier inexistant"
        
        file_size = filepath.stat().st_size
        if expected_size and file_size != expected_size:
            return False, f"Taille incorrecte: {file_size} vs {expected_size}"
        
        if expected_hash:
            with open(filepath, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            if file_hash != expected_hash:
                return False, f"Hash incorrect: {file_hash} vs {expected_hash}"
        
        return True, "Fichier valide"
    
    def check_existing_files(self):
        """Vérifie si les fichiers AMI sont déjà présents et valides."""
        print("🔍 Vérification des fichiers existants...")
        
        # Fichiers audio essentiels
        required_audio = [
            "ES2002a.Headset-0.wav", "ES2002a.Headset-1.wav", 
            "ES2002a.Headset-2.wav", "ES2002a.Headset-3.wav",
            "ES2002b.Headset-0.wav", "ES2002b.Headset-1.wav",
            "ES2002c.Headset-0.wav", "ES2002c.Headset-1.wav",
            "ES2002d.Headset-0.wav", "ES2002d.Headset-1.wav"
        ]
        
        existing_audio = []
        for audio_file in required_audio:
            audio_path = self.audio_dir / audio_file
            if audio_path.exists() and audio_path.stat().st_size > 1000000:  # >1MB
                existing_audio.append(audio_file)
        
        # Fichiers d'annotations
        annotation_files = list(self.annotation_dir.glob("*.rttm"))
        
        print(f"📁 Fichiers audio trouvés: {len(existing_audio)}/{len(required_audio)}")
        print(f"📁 Fichiers d'annotation trouvés: {len(annotation_files)}")
        
        return {
            'audio_complete': len(existing_audio) >= 8,  # Au moins 8 fichiers audio
            'annotations_complete': len(annotation_files) >= 10,
            'existing_audio': existing_audio,
            'existing_annotations': annotation_files
        }
    
    def download_ami_audio_files(self):
        """Télécharge les fichiers audio AMI essentiels."""
        print("🎵 Téléchargement des fichiers audio AMI...")
        
        # URLs réelles pour les fichiers audio AMI (exemple pour ES2002)
        base_url = "https://groups.inf.ed.ac.uk/ami/AMICorpusAnnotations/amicorpus"
        audio_urls = {
            "ES2002a.Headset-0.wav": f"{base_url}/ES2002a/audio/ES2002a.Headset-0.wav",
            "ES2002a.Headset-1.wav": f"{base_url}/ES2002a/audio/ES2002a.Headset-1.wav",
            "ES2002a.Headset-2.wav": f"{base_url}/ES2002a/audio/ES2002a.Headset-2.wav",
            "ES2002a.Headset-3.wav": f"{base_url}/ES2002a/audio/ES2002a.Headset-3.wav",
            "ES2002b.Headset-0.wav": f"{base_url}/ES2002b/audio/ES2002b.Headset-0.wav",
            "ES2002b.Headset-1.wav": f"{base_url}/ES2002b/audio/ES2002b.Headset-1.wav",
        }
        
        downloaded_files = []
        
        for filename, url in audio_urls.items():
            audio_path = self.audio_dir / filename
            
            if audio_path.exists() and audio_path.stat().st_size > 1000000:
                print(f"✅ Déjà présent: {filename}")
                downloaded_files.append(filename)
                continue
            
            try:
                self.download_with_progress_bar(url, str(audio_path), f"Audio {filename}")
                
                # Vérifier la taille
                if audio_path.stat().st_size > 1000000:
                    downloaded_files.append(filename)
                    print(f"✅ Téléchargé avec succès: {filename}")
                else:
                    print(f"⚠️ Fichier trop petit: {filename}")
                    
            except Exception as e:
                print(f"❌ Erreur pour {filename}: {e}")
        
        return downloaded_files
    
    def download_ami_annotations(self):
        """Télécharge les annotations AMI."""
        print("? Téléchargement des annotations AMI...")
        
        annotation_url = "https://groups.inf.ed.ac.uk/ami/AMICorpusAnnotations/ami_public_manual_1.6.2.zip"
        zip_path = self.download_dir / "ami_annotations.zip"
        
        if not zip_path.exists():
            try:
                self.download_with_progress_bar(
                    annotation_url, 
                    str(zip_path), 
                    "Annotations AMI"
                )
            except Exception as e:
                print(f"❌ Erreur téléchargement annotations: {e}")
                return False
        
        # Extraction
        try:
            print("📦 Extraction des annotations...")
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(self.annotation_dir)
            
            # Chercher les fichiers RTTM extraits
            rttm_files = list(self.annotation_dir.rglob("*.rttm"))
            print(f"✅ Extraction terminée: {len(rttm_files)} fichiers RTTM trouvés")
            return True
            
        except Exception as e:
            print(f"❌ Erreur extraction: {e}")
            return False

# Initialiser le téléchargeur AMI
ami_downloader = AMICorpusDownloader(DATA_DIR)

# Vérifier les fichiers existants
existing_status = ami_downloader.check_existing_files()

print("\n" + "="*60)
print("📊 STATUT DU CORPUS AMI")
print("="*60)

if existing_status['audio_complete'] and existing_status['annotations_complete']:
    print("🎉 Corpus AMI complet déjà présent!")
    print(f"   📁 Audio: {len(existing_status['existing_audio'])} fichiers")
    print(f"   📁 Annotations: {len(existing_status['existing_annotations'])} fichiers")
    print("\n✅ Utilisation des fichiers existants...")
    
else:
    print("⚠️ Corpus AMI incomplet - Téléchargement nécessaire")
    
    # Télécharger les fichiers manquants
    if not existing_status['audio_complete']:
        print("\n🔽 Téléchargement des fichiers audio...")
        downloaded_audio = ami_downloader.download_ami_audio_files()
        print(f"✅ Audio téléchargé: {len(downloaded_audio)} fichiers")
    
    if not existing_status['annotations_complete']:
        print("\n🔽 Téléchargement des annotations...")
        ami_downloader.download_ami_annotations()

# Rapport final
final_status = ami_downloader.check_existing_files()
print("\n" + "="*60)
print("📈 RAPPORT FINAL")
print("="*60)
print(f"🎵 Fichiers audio: {len(final_status['existing_audio'])} disponibles")
print(f"📝 Fichiers annotation: {len(final_status['existing_annotations'])} disponibles")

if final_status['audio_complete']:
    print("✅ Corpus audio prêt pour l'entraînement!")
else:
    print("⚠️ Corpus audio incomplet - Mode démonstration activé")

# Configurer les chemins pour la suite
AUDIO_PATH = ami_downloader.audio_dir
ANNOTATION_PATH = ami_downloader.annotation_dir

print(f"\n📂 Chemins configurés:")
print(f"   🎵 Audio: {AUDIO_PATH}")
print(f"   📝 Annotations: {ANNOTATION_PATH}")

In [ ]:
# Validation finale et préparation des données pour l'entraînement

def validate_ami_corpus():
    """Valide que le corpus AMI est prêt pour l'entraînement."""
    print("? Validation finale du corpus AMI...")
    
    audio_files = list(AUDIO_PATH.glob("*.wav"))
    annotation_files = list(ANNOTATION_PATH.rglob("*.rttm"))
    
    print(f"\n📊 Inventaire final:")
    print(f"   🎵 Fichiers audio: {len(audio_files)}")
    print(f"   📝 Fichiers RTTM: {len(annotation_files)}")
    
    # Afficher quelques exemples
    if audio_files:
        print(f"\n🎵 Exemples audio:")
        for i, audio_file in enumerate(audio_files[:5]):
            size_mb = audio_file.stat().st_size / (1024*1024)
            print(f"   - {audio_file.name} ({size_mb:.1f} MB)")
    
    if annotation_files:
        print(f"\n📝 Exemples annotations:")
        for i, rttm_file in enumerate(annotation_files[:5]):
            lines = len(open(rttm_file).readlines())
            print(f"   - {rttm_file.name} ({lines} segments)")
    
    # Statut global
    corpus_ready = len(audio_files) >= 4 and len(annotation_files) >= 4
    
    if corpus_ready:
        print(f"\n✅ Corpus AMI validé et prêt!")
        print(f"🚀 Prêt pour l'entraînement avec {len(audio_files)} fichiers audio")
    else:
        print(f"\n⚠️ Corpus incomplet - Mode démonstration recommandé")
    
    return {
        'ready': corpus_ready,
        'audio_count': len(audio_files),
        'annotation_count': len(annotation_files),
        'audio_files': audio_files,
        'annotation_files': annotation_files
    }

# Valider le corpus
corpus_status = validate_ami_corpus()

print(f"\n{'='*60}")
print(f"🎯 CORPUS AMI: {'PRÊT' if corpus_status['ready'] else 'PARTIEL'}")
print(f"{'='*60}")

# Configurer les chemins finaux pour l'entraînement
if corpus_status['ready']:
    print("✅ Utilisation du corpus AMI réel")
    FINAL_AUDIO_DIR = AUDIO_PATH
    FINAL_ANNOTATION_DIR = ANNOTATION_PATH
else:
    print("⚠️ Basculement vers données de démonstration")
    # On gardera quand même les fichiers partiels s'ils existent
    FINAL_AUDIO_DIR = AUDIO_PATH if corpus_status['audio_count'] > 0 else DATA_DIR / "demo_audio"
    FINAL_ANNOTATION_DIR = ANNOTATION_PATH if corpus_status['annotation_count'] > 0 else DATA_DIR / "demo_annotations"

print(f"\n📂 Configuration finale:")
print(f"   🎵 Audio: {FINAL_AUDIO_DIR}")
print(f"   📝 Annotations: {FINAL_ANNOTATION_DIR}")
print(f"   📊 Mode: {'Production (AMI)' if corpus_status['ready'] else 'Démonstration/Partiel'}")

### 📊 Division des Données (Train/Eval)

Division stratifiée du corpus AMI selon les bonnes pratiques.

In [ ]:
def create_ami_splits(audio_dir, rttm_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Divise le corpus AMI en ensembles d'entraînement, validation et test.
    
    Args:
        audio_dir: Répertoire des fichiers audio
        rttm_dir: Répertoire des annotations RTTM
        train_ratio: Proportion pour l'entraînement
        val_ratio: Proportion pour la validation
        test_ratio: Proportion pour le test
    
    Returns:
        dict: Dictionnaire avec les listes de fichiers pour chaque split
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Les ratios doivent sommer à 1.0"
    
    # Lister tous les fichiers disponibles
    audio_path = Path(audio_dir)
    rttm_path = Path(rttm_dir)
    
    # Trouver les fichiers audio
    audio_extensions = ['.wav', '.pt', '.flac', '.mp3']
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(list(audio_path.glob(f'*{ext}')))
    
    # Vérifier la correspondance audio-RTTM
    valid_pairs = []
    for audio_file in audio_files:
        base_name = audio_file.stem
        rttm_file = rttm_path / f"{base_name}.rttm"
        
        if rttm_file.exists():
            valid_pairs.append({
                'base_name': base_name,
                'audio_file': str(audio_file),
                'rttm_file': str(rttm_file)
            })
    
    print(f"📊 Trouvé {len(valid_pairs)} paires audio-RTTM valides")
    
    if len(valid_pairs) == 0:
        raise ValueError("Aucune paire audio-RTTM valide trouvée!")
    
    # Mélanger et diviser
    import random
    random.seed(42)  # Pour la reproductibilité
    random.shuffle(valid_pairs)
    
    n_total = len(valid_pairs)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    
    train_files = valid_pairs[:n_train]
    val_files = valid_pairs[n_train:n_train + n_val]
    test_files = valid_pairs[n_train + n_val:]
    
    splits = {
        'train': train_files,
        'validation': val_files,
        'test': test_files
    }
    
    # Statistiques
    print(f"\n📈 Division des données:")
    for split_name, files in splits.items():
        print(f"   {split_name:>10}: {len(files):>3} fichiers ({len(files)/n_total*100:.1f}%)")
    
    # Sauvegarder les splits
    splits_file = DATA_DIR / 'data_splits.json'
    with open(splits_file, 'w') as f:
        json.dump(splits, f, indent=2)
    
    print(f"\n💾 Splits sauvegardés dans: {splits_file}")
    return splits

# Créer les splits
data_splits = create_ami_splits(audio_dir, rttm_dir, 
                               train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

# Vérifier le contenu
print("\n🔍 Exemple de fichiers par split:")
for split_name, files in data_splits.items():
    if files:
        print(f"\n{split_name.upper()}:")
        for i, file_info in enumerate(files[:3]):  # Afficher les 3 premiers
            print(f"  {i+1}. {file_info['base_name']}")
        if len(files) > 3:
            print(f"  ... et {len(files)-3} autres")

## 🛠️ 3. Préparation des Données et Extraction de Caractéristiques

Extraction des caractéristiques multi-canaux: LPS, IPD, AF

In [ ]:
# Configuration du path et import des modules du projet
import sys
import os
from pathlib import Path

# Ajouter le répertoire src au path Python
current_dir = Path.cwd()
src_path = current_dir / 'src'

print(f"📁 Répertoire courant: {current_dir}")
print(f"📂 Répertoire src: {src_path}")
print(f"📂 Vérification existence src: {src_path.exists()}")

# Vérifier si le répertoire src existe
if not src_path.exists():
    print("❌ ERREUR: Le répertoire 'src' n'existe pas!")
    print("🔧 Assurez-vous d'avoir cloné le repository complet avec:")
    print("   !git clone https://github.com/saito1111/Speaker-diarization-.git")
    print("   %cd Speaker-diarization-")
    raise FileNotFoundError("Répertoire 'src' manquant. Clonez d'abord le repository.")

# Lister les fichiers dans src pour vérification
py_files = list(src_path.glob("*.py"))
print(f"📄 Fichiers Python trouvés: {[f.name for f in py_files]}")

# Vérifier que TOUS les modules requis existent
required_modules = [
    'tcn_diarization_model.py',
    'metrics.py', 
    'dataset.py',
    'diarization_losses.py',
    'improved_trainer.py',
    'optimized_dataloader.py',
    'optimized_dataset.py'
]

missing_modules = []
for module in required_modules:
    if not (src_path / module).exists():
        missing_modules.append(module)

if missing_modules:
    print(f"❌ ERREUR: Modules manquants dans src/: {missing_modules}")
    print("🔧 Vérifiez que tous les fichiers sont présents dans le repository")
    raise FileNotFoundError(f"Modules manquants: {missing_modules}")

# Ajouter src au path
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"✅ Répertoire src ajouté au path: {src_path}")

# Import de TOUS les modules du projet (OBLIGATOIRES - pas de fallback)
print("\n🔄 Import de TOUS les modules du projet...")

try:
    from tcn_diarization_model import DiarizationTCN
    print("✅ tcn_diarization_model importé")
except ImportError as e:
    print(f"❌ ERREUR CRITIQUE: Impossible d'importer tcn_diarization_model")
    print(f"   Erreur: {e}")
    print("🔧 Vérifiez le contenu du fichier tcn_diarization_model.py")
    raise ImportError("Module tcn_diarization_model requis")

try:
    from metrics import DiarizationMetrics
    print("✅ metrics importé")
except ImportError as e:
    print(f"❌ ERREUR CRITIQUE: Impossible d'importer metrics")
    print(f"   Erreur: {e}")
    print("🔧 Vérifiez le contenu du fichier metrics.py")
    raise ImportError("Module metrics requis")

try:
    from dataset import DiarizationDataset
    print("✅ dataset importé")
except ImportError as e:
    print(f"❌ ERREUR CRITIQUE: Impossible d'importer dataset")
    print(f"   Erreur: {e}")
    print("🔧 Vérifiez le contenu du fichier dataset.py")
    raise ImportError("Module dataset requis")

try:
    from diarization_losses import MultiTaskDiarizationLoss
    print("✅ diarization_losses importé")
except ImportError as e:
    print(f"❌ ERREUR CRITIQUE: Impossible d'importer diarization_losses")
    print(f"   Erreur: {e}")
    print("🔧 Vérifiez le contenu du fichier diarization_losses.py")
    raise ImportError("Module diarization_losses requis")

try:
    from improved_trainer import ImprovedDiarizationTrainer
    print("✅ improved_trainer importé")
except ImportError as e:
    print(f"❌ ERREUR CRITIQUE: Impossible d'importer improved_trainer")
    print(f"   Erreur: {e}")
    print("🔧 Vérifiez le contenu du fichier improved_trainer.py")
    raise ImportError("Module improved_trainer requis")

try:
    from optimized_dataloader import create_optimized_dataloaders
    print("✅ optimized_dataloader importé")
except ImportError as e:
    print(f"❌ ERREUR CRITIQUE: Impossible d'importer optimized_dataloader")
    print(f"   Erreur: {e}")
    print("🔧 Vérifiez le contenu du fichier optimized_dataloader.py")
    raise ImportError("Module optimized_dataloader requis")

try:
    from optimized_dataset import OptimizedDiarizationDataset
    print("✅ optimized_dataset importé")
except ImportError as e:
    print(f"❌ ERREUR CRITIQUE: Impossible d'importer optimized_dataset")
    print(f"   Erreur: {e}")
    print("🔧 Vérifiez le contenu du fichier optimized_dataset.py")
    raise ImportError("Module optimized_dataset requis")

print("\n🎉 TOUS LES MODULES IMPORTÉS AVEC SUCCÈS!")
print("📋 Modules disponibles pour l'entraînement:")
print("   - DiarizationTCN: ✅")
print("   - DiarizationMetrics: ✅")
print("   - DiarizationDataset: ✅")
print("   - MultiTaskDiarizationLoss: ✅")
print("   - ImprovedDiarizationTrainer: ✅")
print("   - OptimizedDataLoader: ✅")
print("   - OptimizedDataset: ✅")

print("\n🚀 Prêt pour l'entraînement avec TOUS vos modèles originaux!")

In [ ]:
# Classes utilitaires nécessaires (non présentes dans les modules du projet)
import numpy as np
import torch
import psutil
import gc

# MemoryMonitor pour surveiller l'utilisation mémoire
class MemoryMonitor:
    def __init__(self):
        self.process = psutil.Process()
        
    def get_memory_info(self):
        """Retourne les informations mémoire."""
        # RAM
        ram_info = psutil.virtual_memory()
        ram_percent = ram_info.percent
        
        # GPU
        gpu_percent = 0
        gpu_memory_used = 0
        gpu_memory_total = 0
        
        if torch.cuda.is_available():
            gpu_memory_used = torch.cuda.memory_allocated(0)
            gpu_memory_total = torch.cuda.get_device_properties(0).total_memory
            gpu_percent = (gpu_memory_used / gpu_memory_total) * 100
        
        return {
            'ram_percent': ram_percent,
            'gpu_percent': gpu_percent,
            'gpu_memory_used': gpu_memory_used,
            'gpu_memory_total': gpu_memory_total
        }
    
    def cleanup(self):
        """Nettoie la mémoire."""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

print("✅ Classes utilitaires créées (MemoryMonitor)")

In [ ]:
# Test de l'extraction de caractéristiques
print("🧪 Test de l'extraction de caractéristiques...")

# Créer un extracteur de caractéristiques
feature_extractor = AudioFeatureExtractor(
    sample_rate=16000,
    n_fft=512,
    hop_length=256
)

# Créer des données audio fictives (8 canaux)
n_channels = 8
duration = 4.0  # 4 secondes
sample_rate = 16000
n_samples = int(duration * sample_rate)

# Simuler audio multi-canal avec du bruit et des signaux
waveforms = []
for ch in range(n_channels):
    # Signal de base + bruit
    base_signal = np.sin(2 * np.pi * 440 * np.linspace(0, duration, n_samples))  # 440 Hz
    noise = np.random.normal(0, 0.1, n_samples)
    # Ajouter un léger décalage temporel pour simuler la spatialisation
    delay_samples = int(0.001 * ch * sample_rate)  # 1ms de délai par canal
    delayed_signal = np.zeros(n_samples)
    if delay_samples < n_samples:
        delayed_signal[delay_samples:] = base_signal[:n_samples-delay_samples]
    
    final_signal = delayed_signal + noise
    waveforms.append(torch.tensor(final_signal, dtype=torch.float32))

print(f"📊 Audio généré: {n_channels} canaux, {duration}s, {sample_rate} Hz")

# Test d'extraction
features = feature_extractor.extract_features(waveforms)
print(f"✅ Caractéristiques extraites: {features.shape}")
print(f"   - Dimensions attendues: [771, ~{int(duration * sample_rate / 256)}]")
print(f"   - Dimensions obtenues: {list(features.shape)}")

# Analyser les caractéristiques
print(f"\n📈 Statistiques des caractéristiques:")
print(f"   - Min: {features.min():.3f}")
print(f"   - Max: {features.max():.3f}")
print(f"   - Moyenne: {features.mean():.3f}")
print(f"   - Std: {features.std():.3f}")

# Visualiser les caractéristiques
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))

# Spectrogramme des caractéristiques
plt.subplot(2, 2, 1)
plt.imshow(features.numpy()[:100, :], aspect='auto', origin='lower')
plt.title('Caractéristiques LPS (première partie)')
plt.xlabel('Temps (frames)')
plt.ylabel('Fréquence')
plt.colorbar()

plt.subplot(2, 2, 2)
plt.imshow(features.numpy()[257:357, :], aspect='auto', origin='lower')
plt.title('Caractéristiques IPD (première partie)')
plt.xlabel('Temps (frames)')
plt.ylabel('Fréquence')
plt.colorbar()

plt.subplot(2, 2, 3)
plt.imshow(features.numpy()[500:600, :], aspect='auto', origin='lower')
plt.title('Caractéristiques AF (première partie)')
plt.xlabel('Temps (frames)')
plt.ylabel('Fréquence')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.plot(features.numpy().mean(axis=0))
plt.title('Énergie moyenne par frame')
plt.xlabel('Temps (frames)')
plt.ylabel('Énergie moyenne')

plt.tight_layout()

# Créer le répertoire de résultats s'il n'existe pas
import os
from pathlib import Path
RESULTS_DIR = Path('./results')
RESULTS_DIR.mkdir(exist_ok=True)

plt.savefig(RESULTS_DIR / 'feature_extraction_test.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Test d'extraction de caractéristiques terminé!")

## 🧠 4. Configuration du Modèle et Entraînement

Configuration optimale pour le corpus AMI avec les meilleures pratiques.

In [ ]:
# Configuration optimisée pour AMI corpus
config = {
    # === MODÈLE ===
    'model': {
        'input_dim': 771,  # LPS (257) + IPD (257*4) + AF (257*4) = 2313 → 771 après agrégation
        'hidden_channels': [256, 256, 256, 512, 512],  # Architecture TCN multi-échelle
        'kernel_size': 3,
        'num_speakers': 4,  # AMI corpus a typiquement 3-4 locuteurs
        'dropout': 0.2,
        'use_attention': True,  # Auto-attention pour dépendances long-terme
        'use_speaker_classifier': True,  # Classification de locuteurs
        'embedding_dim': 256
    },
    
    # === FONCTION DE PERTE ===
    'loss': {
        'type': 'multitask',
        'vad_weight': 1.0,
        'osd_weight': 1.0,
        'consistency_weight': 0.1,
        'use_pit': True,  # Permutation Invariant Training
        'use_focal': True,  # Focal Loss pour données déséquilibrées
        'focal_gamma': 2.0,
        'num_speakers': 4
    },
    
    # === OPTIMISEUR ===
    'optimizer': {
        'type': 'adamw',
        'lr': 1e-3,  # Learning rate initial
        'weight_decay': 1e-4,
        'betas': (0.9, 0.999)
    },
    
    # === PLANIFICATEUR LR ===
    'scheduler': {
        'type': 'onecycle',  # OneCycleLR pour convergence rapide
        'steps_per_epoch': 100,  # Sera mis à jour automatiquement
        'pct_start': 0.3  # 30% montée, 70% descente
    },
    
    # === ENTRAÎNEMENT ===
    'training': {
        'epochs': 50,  # Réduit pour Colab
        'batch_size': 8,  # Adapté à la mémoire Colab
        'num_workers': 2  # Moins de workers pour éviter les problèmes mémoire
    },
    
    # === DONNÉES ===
    'data': {
        'segment_duration': 4.0,  # Segments de 4 secondes
        'sample_rate': 16000,
        'train_split': 0.7,
        'max_segments': 1000  # Limite pour Colab
    },
    
    # === OPTIMISATIONS AVANCÉES ===
    'accumulation_steps': 4,  # Batch effectif = 8*4 = 32
    'use_amp': True,  # Précision mixte
    'grad_clip_norm': 1.0,
    'patience': 10,
    'save_every': 5,
    
    # === MONITORING ===
    'use_wandb': True,  # Weights & Biases (optionnel)
    'project_name': 'ami-speaker-diarization',
    'memory_threshold': 0.85,  # Gestion mémoire Colab
    'adaptive_batch': True,
    'speaker_loss_weight': 0.5,
    
    # === CHEMINS ===
    'save_dir': str(MODEL_DIR),
    'results_dir': str(RESULTS_DIR)
}

print("⚙️ Configuration créée avec les paramètres suivants:")
print(f"   - Architecture: TCN {config['model']['hidden_channels']}")
print(f"   - Batch size: {config['training']['batch_size']} (effectif: {config['training']['batch_size'] * config['accumulation_steps']})")
print(f"   - Epochs: {config['training']['epochs']}")
print(f"   - Learning rate: {config['optimizer']['lr']}")
print(f"   - Précision mixte: {config['use_amp']}")
print(f"   - Classification locuteurs: {config['model']['use_speaker_classifier']}")

# Sauvegarder la configuration
config_file = MODEL_DIR / 'training_config.json'
with open(config_file, 'w') as f:
    json.dump(config, f, indent=2)

print(f"\n💾 Configuration sauvegardée: {config_file}")

In [ ]:
# Configuration de Weights & Biases (optionnel)
import wandb

use_wandb = config.get('use_wandb', False)

if use_wandb:
    try:
        # Connexion à wandb (nécessite un compte gratuit)
        wandb.login()
        
        # Initialisation du projet
        wandb.init(
            project=config['project_name'],
            config=config,
            name=f"ami-tcn-{torch.cuda.get_device_name(0).replace(' ', '-') if torch.cuda.is_available() else 'cpu'}",
            tags=['ami-corpus', 'tcn', 'multi-channel', 'colab'],
            notes="Entraînement sur corpus AMI avec architecture TCN améliorée"
        )
        
        print("✅ Weights & Biases configuré!")
        print(f"📊 Dashboard: {wandb.run.url}")
        
    except Exception as e:
        print(f"⚠️ Erreur wandb: {e}")
        print("📈 Entraînement sans monitoring wandb...")
        config['use_wandb'] = False
else:
    print("📈 Entraînement sans monitoring wandb (désactivé dans config)")

## 🚀 5. Entraînement du Modèle

Entraînement avec toutes les optimisations: gestion mémoire, precision mixte, accumulation de gradients.

In [ ]:
# Création des DataLoaders optimisés
print("🔄 Création des DataLoaders avec vos modules optimisés...")

# Utiliser la fonction create_optimized_dataloaders importée
train_loader, val_loader = create_optimized_dataloaders(
    audio_dir=audio_dir,
    rttm_dir=rttm_dir,
    batch_size=config['training']['batch_size'],
    train_split=config['data']['train_split'],
    num_workers=config['training']['num_workers'],
    segment_duration=config['data']['segment_duration'],
    sample_rate=config['data']['sample_rate'],
    max_segments=config['data']['max_segments'],
    memory_threshold=config['memory_threshold'],
    adaptive_batch=config['adaptive_batch'],
    accumulation_steps=config['accumulation_steps']
)

print(f"✅ DataLoaders créés avec succès!")
print(f"   - Train batches: {len(train_loader)}")
print(f"   - Validation batches: {len(val_loader)}")

# Mettre à jour la configuration avec le nombre réel de steps
config['scheduler']['steps_per_epoch'] = len(train_loader)

# Test d'un batch
print("\n🧪 Test d'un batch d'entraînement...")
for batch_idx, batch in enumerate(train_loader):
    print(f"   Batch {batch_idx}:")
    print(f"     - Features: {batch['features'].shape}")
    print(f"     - VAD labels: {batch['vad_labels'].shape}")
    print(f"     - OSD labels: {batch['osd_labels'].shape}")
    
    # Vérifier les dimensions
    assert batch['features'].shape[1] == 771, f"Dimension features incorrecte: {batch['features'].shape[1]} != 771"
    assert batch['vad_labels'].shape[-1] == 4, f"Nombre de locuteurs incorrect: {batch['vad_labels'].shape[-1]} != 4"
    
    print(f"     ✅ Dimensions correctes!")
    break

print("✅ DataLoaders optimisés configurés et testés avec succès!")

In [ ]:
# Initialisation du trainer avancé
print("🧠 Initialisation du trainer avec vos modules optimisés...")

# Créer le trainer avec toutes les améliorations (OBLIGATOIRE - pas de fallback)
trainer = ImprovedDiarizationTrainer(config)

print(f"✅ Trainer initialisé avec succès!")
print(f"   - Modèle: {trainer.model.get_num_params():,} paramètres")
print(f"   - Device: {trainer.device}")
print(f"   - Précision mixte: {trainer.use_amp}")
print(f"   - Accumulation gradients: {trainer.accumulation_steps}")

# Test du forward pass
print("\n🧪 Test du modèle...")
model = trainer.model
model.eval()

with torch.no_grad():
    # Test avec un batch de données
    test_input = torch.randn(2, 771, 250).to(trainer.device)
    
    # Forward pass simple
    vad_out, osd_out = model(test_input)
    print(f"   Forward simple: VAD {vad_out.shape}, OSD {osd_out.shape}")
    
    # Forward avec embeddings si disponible
    try:
        vad_out, osd_out, embeddings, speaker_logits = model(test_input, return_embeddings=True)
        print(f"   Forward complet: VAD {vad_out.shape}, OSD {osd_out.shape}")
        print(f"                   Embeddings {embeddings.shape}, Speaker {speaker_logits.shape}")
    except:
        print("   Forward avec embeddings non disponible (normal)")

print("✅ Modèle fonctionne correctement!")
print("🚀 Trainer optimisé prêt pour l'entraînement!")

In [ ]:
# Démarrage de l'entraînement
print("🚀 DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Monitoring mémoire
memory_monitor = MemoryMonitor()
initial_memory = memory_monitor.get_memory_info()

print(f"💾 Mémoire initiale:")
print(f"   - RAM: {initial_memory['ram_percent']:.1f}%")
print(f"   - GPU: {initial_memory['gpu_percent']:.1f}%")

# Configuration d'entraînement
num_epochs = config['training']['epochs']
save_every = config.get('save_every', 5)

# Historique des métriques
train_losses = []
val_losses = []
best_val_loss = float('inf')

print(f"\n📋 Configuration d'entraînement:")
print(f"   - Epochs: {num_epochs}")
print(f"   - Batch size: {config['training']['batch_size']}")
print(f"   - Learning rate: {config['optimizer']['lr']}")
print(f"   - Sauvegarde chaque {save_every} epochs")

print("\n" + "="*50)
print("DÉBUT DE L'ENTRAÎNEMENT")
print("="*50)

import time
from datetime import datetime

# Boucle d'entraînement principale
start_time = time.time()

try:
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        print(f"\n🔄 Epoch {epoch+1}/{num_epochs} - {datetime.now().strftime('%H:%M:%S')}")
        print("-" * 40)
        
        # Phase d'entraînement
        if hasattr(trainer, 'train_epoch'):
            # Utiliser le trainer avancé
            train_metrics = trainer.train_epoch(train_loader)
            train_loss = train_metrics['total_loss']
        else:
            # Entraînement simple
            trainer.model.train()
            total_loss = 0
            num_batches = 0
            
            for batch_idx, batch in enumerate(train_loader):
                features = batch['features'].to(trainer.device)
                vad_labels = batch['vad_labels'].to(trainer.device)
                osd_labels = batch['osd_labels'].to(trainer.device)
                
                trainer.optimizer.zero_grad()
                
                vad_pred, osd_pred = trainer.model(features)
                loss_dict = trainer.criterion(vad_pred, osd_pred, vad_labels, osd_labels)
                loss = loss_dict['total_loss']
                
                loss.backward()
                trainer.optimizer.step()
                
                total_loss += loss.item()
                num_batches += 1
                
                if batch_idx % 20 == 0:
                    print(f"   Batch {batch_idx}/{len(train_loader)}: Loss {loss.item():.4f}")
            
            train_loss = total_loss / num_batches
        
        train_losses.append(train_loss)
        
        # Phase de validation
        print(f"\n📊 Validation...")
        trainer.model.eval()
        val_loss = 0
        num_val_batches = 0
        
        with torch.no_grad():
            for batch in val_loader:
                features = batch['features'].to(trainer.device)
                vad_labels = batch['vad_labels'].to(trainer.device)
                osd_labels = batch['osd_labels'].to(trainer.device)
                
                vad_pred, osd_pred = trainer.model(features)
                loss_dict = trainer.criterion(vad_pred, osd_pred, vad_labels, osd_labels)
                val_loss += loss_dict['total_loss'].item()
                num_val_batches += 1
        
        val_loss = val_loss / num_val_batches if num_val_batches > 0 else 0
        val_losses.append(val_loss)
        
        # Monitoring mémoire
        current_memory = memory_monitor.get_memory_info()
        
        # Résumé de l'epoch
        epoch_time = time.time() - epoch_start
        print(f"\n📈 Epoch {epoch+1} Résultats:")
        print(f"   - Train Loss: {train_loss:.4f}")
        print(f"   - Val Loss: {val_loss:.4f}")
        print(f"   - Temps: {epoch_time:.1f}s")
        print(f"   - Mémoire GPU: {current_memory['gpu_percent']:.1f}%")
        
        # Sauvegarde du meilleur modèle
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = MODEL_DIR / 'best_model.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': trainer.model.state_dict(),
                'optimizer_state_dict': trainer.optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'config': config
            }, best_model_path)
            print(f"   ✅ Nouveau meilleur modèle sauvé! (Loss: {val_loss:.4f})")
        
        # Sauvegarde périodique
        if (epoch + 1) % save_every == 0:
            checkpoint_path = MODEL_DIR / f'checkpoint_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': trainer.model.state_dict(),
                'optimizer_state_dict': trainer.optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_losses': train_losses,
                'val_losses': val_losses,
                'config': config
            }, checkpoint_path)
            print(f"   💾 Checkpoint sauvé: {checkpoint_path.name}")
        
        # Logging wandb
        if config.get('use_wandb', False) and 'wandb' in locals():
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss,
                'val_loss': val_loss,
                'best_val_loss': best_val_loss,
                'gpu_memory_percent': current_memory['gpu_percent'],
                'epoch_time': epoch_time
            })
        
        # Nettoyage mémoire
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

except KeyboardInterrupt:
    print("\n⏹️ Entraînement interrompu par l'utilisateur")
except Exception as e:
    print(f"\n❌ Erreur durant l'entraînement: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    total_time = time.time() - start_time
    print(f"\n⏱️ Temps total d'entraînement: {total_time/60:.1f} minutes")
    print(f"🎯 Meilleure validation loss: {best_val_loss:.4f}")
    
    # Fermeture wandb
    if config.get('use_wandb', False) and 'wandb' in locals():
        wandb.finish()
    
print("\n" + "="*50)
print("ENTRAÎNEMENT TERMINÉ")
print("="*50)

## 📊 6. Évaluation et Métriques

Évaluation complète avec métriques de diarization standard.

In [ ]:
# Évaluation complète du modèle
print("📊 ÉVALUATION DU MODÈLE")
print("=" * 40)

# Charger le meilleur modèle
best_model_path = MODEL_DIR / 'best_model.pth'

if best_model_path.exists():
    print(f"📂 Chargement du meilleur modèle: {best_model_path}")
    checkpoint = torch.load(best_model_path, map_location=trainer.device)
    trainer.model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"   - Epoch: {checkpoint['epoch']}")
    print(f"   - Val Loss: {checkpoint['val_loss']:.4f}")
else:
    print("⚠️ Pas de modèle sauvé, utilisation du modèle actuel")

# Initialiser les métriques
metrics_computer = DiarizationMetrics(num_speakers=config['model']['num_speakers'])

# Évaluation sur l'ensemble de validation
print("\n🧪 Évaluation sur l'ensemble de validation...")
trainer.model.eval()

all_vad_preds = []
all_vad_targets = []
all_osd_preds = []
all_osd_targets = []

eval_loss = 0
num_eval_batches = 0

with torch.no_grad():
    for batch_idx, batch in enumerate(val_loader):
        features = batch['features'].to(trainer.device)
        vad_labels = batch['vad_labels'].to(trainer.device)
        osd_labels = batch['osd_labels'].to(trainer.device)
        
        # Prédictions
        if hasattr(trainer.model, 'use_speaker_classifier') and trainer.model.use_speaker_classifier:
            vad_pred, osd_pred, embeddings, speaker_logits = trainer.model(features, return_embeddings=True)
        else:
            vad_pred, osd_pred = trainer.model(features)
        
        # Loss
        loss_dict = trainer.criterion(vad_pred, osd_pred, vad_labels, osd_labels)
        eval_loss += loss_dict['total_loss'].item()
        num_eval_batches += 1
        
        # Collecter pour métriques
        all_vad_preds.append(vad_pred.cpu())
        all_vad_targets.append(vad_labels.cpu())
        all_osd_preds.append(osd_pred.cpu())
        all_osd_targets.append(osd_labels.cpu())
        
        if batch_idx % 10 == 0:
            print(f"   Batch {batch_idx}/{len(val_loader)} évalué")

# Calculer métriques détaillées
print("\n📈 Calcul des métriques détaillées...")

vad_preds = torch.cat(all_vad_preds, dim=0)
vad_targets = torch.cat(all_vad_targets, dim=0)
osd_preds = torch.cat(all_osd_preds, dim=0)
osd_targets = torch.cat(all_osd_targets, dim=0)

print(f"   - Données évaluées: {vad_preds.shape[0]} échantillons")
print(f"   - Durée totale: {vad_preds.shape[0] * vad_preds.shape[1] * 0.02:.1f} secondes")

# Métriques principales
metrics = metrics_computer.compute_metrics(vad_preds, osd_preds, vad_targets, osd_targets)

print("\n🎯 RÉSULTATS D'ÉVALUATION")
print("=" * 30)
print(f"📊 Loss finale: {eval_loss / num_eval_batches:.4f}")
print(f"📊 DER (Diarization Error Rate): {metrics.get('der', 0):.2f}%")
print(f"📊 F1 Score global: {metrics.get('f1_score', 0):.3f}")
print(f"📊 Précision frame: {metrics.get('frame_precision', 0):.3f}")
print(f"📊 Rappel frame: {metrics.get('frame_recall', 0):.3f}")
print(f"📊 Jaccard Index: {metrics.get('jaccard_index', 0):.3f}")

# Métriques OSD
if 'osd_precision' in metrics:
    print(f"\n🔀 Détection de Chevauchement (OSD):")
    print(f"   - Précision OSD: {metrics['osd_precision']:.3f}")
    print(f"   - Rappel OSD: {metrics['osd_recall']:.3f}")
    print(f"   - F1 OSD: {metrics['osd_f1']:.3f}")

# Métriques par locuteur
print(f"\n👥 Métriques par Locuteur:")
for spk in range(config['model']['num_speakers']):
    if f'speaker_{spk}_f1' in metrics:
        print(f"   Locuteur {spk}: F1={metrics[f'speaker_{spk}_f1']:.3f}, "
              f"P={metrics[f'speaker_{spk}_precision']:.3f}, "
              f"R={metrics[f'speaker_{spk}_recall']:.3f}")

# Sauvegarder les métriques
metrics_file = RESULTS_DIR / 'evaluation_metrics.json'
with open(metrics_file, 'w') as f:
    # Convertir les tenseurs en listes pour JSON
    json_metrics = {k: (v.item() if torch.is_tensor(v) else v) for k, v in metrics.items()}
    json_metrics['eval_loss'] = eval_loss / num_eval_batches
    json.dump(json_metrics, f, indent=2)

print(f"\n💾 Métriques sauvées: {metrics_file}")

## 📈 7. Visualisations et Analyses

Génération de graphiques et visualisations des résultats.

In [ ]:
# Visualisations des résultats
print("📊 Génération des visualisations...")

# Configuration matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

# === 1. Courbes d'entraînement ===
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Résultats d\'Entraînement - Speaker Diarization TCN', fontsize=16, fontweight='bold')

# Courbe de perte
axes[0, 0].plot(train_losses, label='Train Loss', color='blue', linewidth=2)
axes[0, 0].plot(val_losses, label='Validation Loss', color='red', linewidth=2)
axes[0, 0].set_title('Évolution de la Perte')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Zoom sur les dernières epochs
if len(train_losses) > 10:
    start_idx = max(0, len(train_losses) - 20)
    axes[0, 1].plot(range(start_idx, len(train_losses)), train_losses[start_idx:], 
                   label='Train Loss', color='blue', linewidth=2)
    axes[0, 1].plot(range(start_idx, len(val_losses)), val_losses[start_idx:], 
                   label='Validation Loss', color='red', linewidth=2)
    axes[0, 1].set_title('Convergence (Dernières Epochs)')
else:
    axes[0, 1].plot(train_losses, label='Train Loss', color='blue', linewidth=2)
    axes[0, 1].plot(val_losses, label='Validation Loss', color='red', linewidth=2)
    axes[0, 1].set_title('Évolution de la Perte (Toutes Epochs)')

axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# === 2. Analyse des prédictions ===
# Prendre un échantillon pour visualisation
sample_idx = 0
sample_vad_pred = vad_preds[sample_idx].numpy()  # [time, speakers]
sample_vad_target = vad_targets[sample_idx].numpy()
sample_osd_pred = osd_preds[sample_idx].numpy()  # [time]
sample_osd_target = osd_targets[sample_idx].numpy()

# Activité des locuteurs (prédictions vs vérité terrain)
time_frames = np.arange(len(sample_vad_pred)) * 0.02  # Conversion en secondes

# Subplot pour VAD
axes[1, 0].imshow(sample_vad_pred.T, aspect='auto', origin='lower', 
                 extent=[0, len(sample_vad_pred)*0.02, 0, 4], 
                 cmap='Blues', alpha=0.7)
axes[1, 0].imshow(sample_vad_target.T, aspect='auto', origin='lower',
                 extent=[0, len(sample_vad_target)*0.02, 0, 4],
                 cmap='Reds', alpha=0.5)
axes[1, 0].set_title('Activité VAD: Prédiction (Bleu) vs Vérité (Rouge)')
axes[1, 0].set_xlabel('Temps (s)')
axes[1, 0].set_ylabel('Locuteur ID')
axes[1, 0].set_yticks(range(4))

# Subplot pour OSD
axes[1, 1].plot(time_frames, sample_osd_pred, label='Prédiction OSD', 
               color='blue', linewidth=2, alpha=0.8)
axes[1, 1].plot(time_frames, sample_osd_target, label='Vérité OSD', 
               color='red', linewidth=2, alpha=0.6)
axes[1, 1].set_title('Détection de Chevauchement (OSD)')
axes[1, 1].set_xlabel('Temps (s)')
axes[1, 1].set_ylabel('Probabilité de Chevauchement')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(0, 1)

plt.tight_layout()
training_plot_path = RESULTS_DIR / 'training_results.png'
plt.savefig(training_plot_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphique d'entraînement sauvé: {training_plot_path}")

In [ ]:
# === 3. Matrice de confusion pour classification ===
if hasattr(trainer.model, 'use_speaker_classifier') and trainer.model.use_speaker_classifier:
    print("\n📊 Analyse de la classification des locuteurs...")
    
    # Extraire les prédictions de classification
    all_speaker_preds = []
    all_speaker_targets = []
    
    trainer.model.eval()
    with torch.no_grad():
        for batch in val_loader:
            features = batch['features'].to(trainer.device)
            vad_labels = batch['vad_labels'].to(trainer.device)
            
            try:
                vad_pred, osd_pred, embeddings, speaker_logits = trainer.model(features, return_embeddings=True)
                
                # Créer des labels de locuteurs à partir des VAD labels
                speaker_targets = torch.argmax(vad_labels.sum(dim=1), dim=1)  # Locuteur le plus actif
                speaker_preds = torch.argmax(speaker_logits, dim=1)
                
                all_speaker_preds.extend(speaker_preds.cpu().numpy())
                all_speaker_targets.extend(speaker_targets.cpu().numpy())
                
            except Exception as e:
                print(f"   Erreur dans un batch: {e}")
                continue
    
    if all_speaker_preds and all_speaker_targets:
        from sklearn.metrics import confusion_matrix, classification_report
        
        # Matrice de confusion
        cm = confusion_matrix(all_speaker_targets, all_speaker_preds)
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=[f'Pred {i}' for i in range(4)],
                   yticklabels=[f'True {i}' for i in range(4)])
        plt.title('Matrice de Confusion - Classification des Locuteurs', fontsize=14, fontweight='bold')
        plt.xlabel('Prédiction')
        plt.ylabel('Vérité Terrain')
        
        confusion_path = RESULTS_DIR / 'speaker_confusion_matrix.png'
        plt.savefig(confusion_path, dpi=300, bbox_inches='tight')
        plt.show()
        
        # Rapport de classification
        print("\n📋 Rapport de Classification:")
        print(classification_report(all_speaker_targets, all_speaker_preds,
                                  target_names=[f'Locuteur {i}' for i in range(4)],
                                  digits=3))
        
        print(f"✅ Matrice de confusion sauvée: {confusion_path}")
    else:
        print("⚠️ Pas assez de données pour la matrice de confusion")
else:
    print("⚠️ Classificateur de locuteurs non activé")

In [ ]:
# === 4. Résumé final et comparaisons ===
print("\n" + "="*60)
print("📊 RÉSUMÉ FINAL DE L'ENTRAÎNEMENT")
print("="*60)

# Créer un résumé complet
final_summary = {
    'Configuration': {
        'Architecture': f"TCN {config['model']['hidden_channels']}",
        'Paramètres': f"{sum(p.numel() for p in trainer.model.parameters()):,}",
        'Batch Size': config['training']['batch_size'],
        'Epochs': len(train_losses),
        'Learning Rate': config['optimizer']['lr'],
        'Device': str(trainer.device)
    },
    'Résultats Finaux': {
        'Train Loss': f"{train_losses[-1]:.4f}" if train_losses else "N/A",
        'Val Loss': f"{val_losses[-1]:.4f}" if val_losses else "N/A",
        'Best Val Loss': f"{best_val_loss:.4f}",
        'DER': f"{metrics.get('der', 0):.2f}%",
        'F1 Score': f"{metrics.get('f1_score', 0):.3f}",
        'Frame Precision': f"{metrics.get('frame_precision', 0):.3f}",
        'Frame Recall': f"{metrics.get('frame_recall', 0):.3f}"
    },
    'Fichiers Générés': {
        'Meilleur Modèle': str(best_model_path) if best_model_path.exists() else "Non sauvé",
        'Métriques': str(metrics_file),
        'Graphiques': str(training_plot_path),
        'Configuration': str(config_file)
    }
}

# Affichage du résumé
for section, items in final_summary.items():
    print(f"\n🔹 {section}:")
    for key, value in items.items():
        print(f"   {key:.<25} {value}")

# Sauvegarde du résumé
summary_file = RESULTS_DIR / 'training_summary.json'
with open(summary_file, 'w') as f:
    json.dump(final_summary, f, indent=2)

print(f"\n💾 Résumé complet sauvé: {summary_file}")

# === 5. Recommandations d'amélioration ===
print("\n" + "="*60)
print("💡 RECOMMANDATIONS POUR AMÉLIORER LES PERFORMANCES")
print("="*60)

current_der = metrics.get('der', 100)
current_f1 = metrics.get('f1_score', 0)

recommendations = []

if current_der > 25:
    recommendations.append("🔧 DER élevé: Augmenter le nombre d'epochs ou réduire le learning rate")
if current_f1 < 0.7:
    recommendations.append("🔧 F1 faible: Essayer focal loss avec gamma plus élevé")
if len(train_losses) < 20:
    recommendations.append("⏰ Entraînement court: Augmenter le nombre d'epochs")
if config['training']['batch_size'] < 16:
    recommendations.append("📦 Batch size petit: Augmenter si possible pour améliorer la stabilité")

recommendations.extend([
    "📊 Utiliser plus de données AMI si disponibles",
    "🎯 Affiner les hyperparamètres avec Optuna",
    "🔄 Essayer l'ensemble de modèles",
    "📈 Implémenter la validation croisée",
    "🧠 Tester différentes architectures d'attention"
])

for i, rec in enumerate(recommendations[:7], 1):
    print(f"{i}. {rec}")

print("\n" + "="*60)
print("🎉 ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS!")
print("="*60)
print(f"📁 Tous les résultats sont sauvés dans: {RESULTS_DIR}")
print(f"🧠 Meilleur modèle disponible dans: {MODEL_DIR}")

if config.get('use_wandb', False):
    print(f"📊 Logs détaillés disponibles sur Weights & Biases")

## 🚀 Prochaines Étapes

### 📝 Pour Continuer l'Amélioration:

1. **📊 Données**: Télécharger le corpus AMI complet
2. **⚙️ Hyperparamètres**: Optimiser avec Optuna
3. **🎯 Architecture**: Tester différentes tailles de modèle
4. **📈 Ensembles**: Combiner plusieurs modèles
5. **🔄 Post-traitement**: Améliorer la segmentation finale

### 💾 Fichiers Générés:
- `models/checkpoints/best_model.pth` - Meilleur modèle
- `results/evaluation_metrics.json` - Métriques détaillées
- `results/training_results.png` - Graphiques d'entraînement
- `results/training_summary.json` - Résumé complet

### 🎯 Objectifs de Performance:
- **DER < 20%** sur AMI corpus (état de l'art: ~15-18%)
- **F1 > 0.8** pour la détection d'activité
- **Temps réel** pour l'inférence

---

**🎉 Félicitations! Vous avez entraîné avec succès un système de diarization moderne avec toutes les optimisations avancées!**